In [9]:
import torch
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Load datasets
try:
    ratings_df = pd.read_csv("ratings.csv")
    movies_df = pd.read_csv("movies.csv")
    print("Successfully loaded MovieLens dataset")
except FileNotFoundError:
    print("ERROR: Dataset files not found.")
    exit()

Successfully loaded MovieLens dataset


In [10]:
ratings = {}
for i in range(len(ratings_df)):
    u = ratings_df["userId"][i]
    m = ratings_df["movieId"][i]
    r = ratings_df["rating"][i]
    ratings[(u, m)] = r

data = [(user, movie, rating) for (user, movie), rating in ratings.items()]
train_ratings, test_ratings = train_test_split(data, test_size=0.2, random_state=10)

100836


In [ ]:
# ratings input: list of (i, j, r_ij) tuples
def model(ratings, n_users, m_movies, k):
    # Priors for user latent factors and bias
    with pyro.plate("users", n_users):
        nu = pyro.sample("nu", dist.Normal(0., 0.1))
        U = pyro.sample("U", dist.MultivariateNormal(torch.zeros(k), torch.eye(k)))
    
    # Priors for movie latent factors and bias
    with pyro.plate("movies", m_movies):
        mu = pyro.sample("mu", dist.Normal(0., 0.316))
        V = pyro.sample("V", dist.MultivariateNormal(torch.zeros(k), torch.eye(k)))
    
    sigma = 0.5  # observation noise

    # Plate over observed ratings
    with pyro.plate("observe_data", len(ratings)):
        users = torch.tensor([r[0] for r in ratings])
        items = torch.tensor([r[1] for r in ratings])
        observed_ratings = torch.tensor([r[2] for r in ratings])

        pred = nu[users] + mu[items] + torch.sum(U[users] * V[items], dim=1)
        pyro.sample("obs", dist.Normal(pred, sigma), obs=observed_ratings)